In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time

def scrape_hepsiemlak_for_all_cities(max_batch=1):
    # Chrome ayarları
    options = Options()
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    all_data = []

    all_cities = [
        "adiyaman", "afyonkarahisar", "agri", "aksaray",
        "amasya", "ankara", "antalya", "ardahan", "artvin",
        "aydin", "balikesir", "bartin", "batman", "bayburt",
        "bilecik", "bingol", "bitlis", "bolu", "burdur",
        "bursa", "canakkale", "cankiri", "corum", "denizli",
        "diyarbakir", "duzce", "edirne", "elazig", "erzincan",
        "erzurum", "eskisehir", "gaziantep", "giresun", "gumushane",
        "hakkari", "hatay", "igdir", "isparta", "istanbul",
        "izmir", "kahramanmaras", "karabuk", "karaman", "kars",
        "kastamonu", "kayseri", "kilis", "kirikkale", "kirklareli",
        "kirsehir", "kocaeli", "konya", "kutahya", "malatya",
        "manisa", "mardin", "mersin", "mugla", "mus",
        "nevsehir", "nigde", "ordu", "osmaniye", "rize",
        "sakarya", "samsun", "siirt", "sinop", "sivas",
        "sanliurfa", "sirnak", "tekirdag", "tokat", "trabzon",
        "tunceli", "usak", "van", "yalova", "yozgat", "zonguldak"
    ]

    big_cities = [
        "istanbul", "ankara", "izmir", "bursa", "antalya",
        "konya", "adana", "kocaeli", "mugla", "mersin",
        "manisa", "kayseri", "samsun", "tekirdag", "eskisehir",
        "gaziantep", "sanliurfa", "trabzon", "denizli", "sakarya"
    ]

    for city in all_cities:
        print(f"\n>>> Şehir: {city.capitalize()}")

        base_url = f"https://www.hepsiemlak.com/{city}-satilik"

        page_step = 50 if city in big_cities else 5

        current_page = 1
        batch_count = 0

        while batch_count < max_batch:
            url = f"{base_url}?page={current_page}"
            driver.get(url)
            time.sleep(3)

            soup = BeautifulSoup(driver.page_source, "html.parser")
            listings = soup.find_all("li", class_="listing-item")

            if not listings:
                print(f"Sayfa {current_page}: İlan yok, bu şehirde durduruluyor.")
                break

            print(f"Sayfa {current_page}: {len(listings)} ilan bulundu.")

            for item in listings:
                def safe_select(tag, class_name):
                    found = item.find(class_=class_name)
                    return found.get_text(strip=True) if found else ""

                title = safe_select(item, "list-view-title")
                price = safe_select(item, "list-view-price")
                date = safe_select(item, "list-view-date")
                rooms = safe_select(item, "celly houseRoomCount")
                sqm = safe_select(item, "celly squareMeter")
                building_age = safe_select(item, "celly buildingAge")
                floor = safe_select(item, "celly floortype")
                location = safe_select(item, "list-view-location")
                firm = safe_select(item, "listing-card--owner-info__firm-name")
                agent = safe_select(item, "listing-card--owner-info__name")

                link_tag = item.find("a", class_="card-link")
                link = "https://www.hepsiemlak.com" + link_tag["href"] if link_tag else ""

                img_tag = item.find("img", class_="list-view-image")
                image_url = img_tag["src"] if img_tag else ""

                all_data.append({
                    "city": city.capitalize(),
                    "title": title,
                    "price": price,
                    "date": date,
                    "rooms": rooms,
                    "sqm": sqm,
                    "building_age": building_age,
                    "floor": floor,
                    "location": location,
                    "firm": firm,
                    "agent": agent,
                    "link": link,
                    "image_url": image_url
                })

            current_page += page_step
            batch_count += 1

    driver.quit()
    return all_data

# Çalıştır
results = scrape_hepsiemlak_for_all_cities(max_batch=2)

# Sonuçları yazdır
for idx, r in enumerate(results, 1):
    print(f"\n{idx}. [{r['city']}] {r['title']}")
    print(f"   Fiyat: {r['price']} | Tarih: {r['date']} | Oda: {r['rooms']} | m2: {r['sqm']}")
    print(f"   Kat: {r['floor']} | Yaş: {r['building_age']}")
    print(f"   Lokasyon: {r['location']}")
    print(f"   Emlakçı: {r['firm']} - {r['agent']}")
    print(f"   Link: {r['link']}")
    print(f"   Resim: {r['image_url']}")



>>> Şehir: Adiyaman
Sayfa 1: 24 ilan bulundu.
Sayfa 6: 24 ilan bulundu.

>>> Şehir: Afyonkarahisar
Sayfa 1: 21 ilan bulundu.
Sayfa 6: 24 ilan bulundu.

>>> Şehir: Agri
Sayfa 1: 19 ilan bulundu.
Sayfa 6: 19 ilan bulundu.

>>> Şehir: Aksaray
Sayfa 1: 24 ilan bulundu.
Sayfa 6: 24 ilan bulundu.

>>> Şehir: Amasya
Sayfa 1: 24 ilan bulundu.
Sayfa 6: 24 ilan bulundu.

>>> Şehir: Ankara
Sayfa 1: 19 ilan bulundu.
Sayfa 51: 24 ilan bulundu.

>>> Şehir: Antalya
Sayfa 1: 19 ilan bulundu.
Sayfa 51: 24 ilan bulundu.

>>> Şehir: Ardahan
Sayfa 1: 9 ilan bulundu.
Sayfa 6: 9 ilan bulundu.

>>> Şehir: Artvin
Sayfa 1: 24 ilan bulundu.
Sayfa 6: 24 ilan bulundu.

>>> Şehir: Aydin
Sayfa 1: 19 ilan bulundu.
Sayfa 6: 24 ilan bulundu.

>>> Şehir: Balikesir
Sayfa 1: 21 ilan bulundu.
Sayfa 6: 24 ilan bulundu.

>>> Şehir: Bartin
Sayfa 1: 23 ilan bulundu.
Sayfa 6: 24 ilan bulundu.

>>> Şehir: Batman
Sayfa 1: 24 ilan bulundu.
Sayfa 6: 24 ilan bulundu.

>>> Şehir: Bayburt
Sayfa 1: 9 ilan bulundu.
Sayfa 6: 9 ilan bul